In [1]:
import pandas as pd
import os
import time

In [2]:
data_folder='./Data/'

In [22]:
file_channels = open(data_folder + "channel_ids.txt", "r")
file_pagerank_all= open(data_folder + "pagerank_all.txt", "r")
file_pagerank_2018= open(data_folder + "pagerank_2018_2019.txt", "r")

data_channels = file_channels.read()
data_pagerank_all = file_pagerank_all.read()
data_pagerank_2018 = file_pagerank_2018.read()

list_channels = data_channels.split("\n")
list_channels = list_channels[:-1] #discrading the last entry as it contains no information (residual from the split)
list_pagerank_all = data_pagerank_all.split("\n")
list_pagerank_all= list_pagerank_all[3:-2]#discrading first 3 values and the last one as they are not channel
#and the one before the last one as it doesn't contain anything (residual from the split)
list_pagerank_2018 = data_pagerank_2018.split("\n")
list_pagerank_2018= list_pagerank_2018[3:-2]#discrading first 3 values and the last one as they are not channel
#and the one before the last one as it doesn't contain anything (residual from the split)

In [23]:
print(len(list_channels), len(list_pagerank_all), len(list_pagerank_2018)) #all good the lengths match

136470 136470 136470


In [25]:
#creating a dataframe
zipped = list(zip(list_channels, list_pagerank_all, list_pagerank_2018))
df_cluster=pd.DataFrame(zipped, columns=['channel_id', 'pagerank', 'pagerank_2018'])

In [26]:
#notice that the way our data was stored was so that if we stack them next to each other, each line had a channel_id and
#the channel's associated pagerank cluster coefficient for all videos and the associated pagerank cluster coefficient
#for the time window 2018-2019
df_cluster

,channel_id,pagerank,pagerank_2018
0,UC-lHJZR3Gqxm24_Vd_AJ5Yw,1.7655139160156250e+02,1.9549287414550781e+02
1,UCbCmjCuTUZos6Inko4u57UQ,8.9536696672439575e-01,1.3123830556869507e+00
2,UCpEhnqL0y41EpW2TvWAHD7Q,6.8004007339477539e+00,6.4472532272338867e+00
3,UC295-Dw_tDNtZXFeAPAW6Aw,2.8799509048461914e+01,3.7452735900878906e+01
4,UCJ5v_MCY6GNUBTO8-D3XoAg,3.2658580780029297e+01,3.2937046051025391e+01
...,...,...,...
136465,UCuM-9AajUOwKw6ipOzu2DRQ,5.0469939596951008e-03,9.0959090739488602e-03
136466,UCtW9jp5TH0YrgYpwiRf9t-Q,1.4429617673158646e-02,4.3094335123896599e-03
136467,UCTsxFTIUs8vFDzGccDm6i7Q,1.8618732690811157e-01,2.6001319289207458e-01
136468,UC1HOArgRCMGPjlcmkThERwA,8.2442117854952812e-03,0.0000000000000000e+00


In [34]:
path_sources_final='sources_final.csv'
df_sources_final=pd.read_csv(data_folder + path_sources_final) #maps channel_id to extreme grp corresponding
df_sources=df_sources_final[["Category", "Id"]]
df_sources.rename(columns = {'Id':'channel_id'}, inplace = True)

C:\Users\HOURS\AppData\Local\Temp\ipykernel_5984\100672720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sources.rename(columns = {'Id':'channel_id'}, inplace = True)


In [37]:
df_sources_interest=df_sources[df_sources['Category'].isin(['Alt-lite', 'Alt-right', 'Intellectual Dark Web'])]

In [40]:
df_sources_interest.columns

Index(['Category', 'channel_id'], dtype='object')

In [42]:
df_sources_interest['Category'].value_counts()

Alt-lite                 113
Intellectual Dark Web     89
Alt-right                 88
Name: Category, dtype: int64

In [ ]:
Alt-lite                 112
Intellectual Dark Web     86
Alt-right                 81

In [39]:
len(df_sources_interest)

290

In [30]:
#page rank coeff for all and only 2018-2019 for only the channels of interest
df_cluster_interest=df_cluster[df_cluster['channel_id'].isin(list_channels_interest)]

In [ ]:
df_cluster_interest['']

In [47]:
df_cluster_interest=df_cluster.merge(df_sources_interest, how='right', on='channel_id')
df_cluster_interest

,channel_id,pagerank,pagerank_2018,Category
0,UCfDdlNLRVb1h3_7Xh-WhL3w,1.1852910041809082e+01,2.2547094345092773e+01,Alt-lite
1,UCso6oQ3qy7z61Fr8b0WNBMw,NaN,NaN,Alt-lite
2,UCplXdu0g94UynzYCmO2zOjw,5.0285243988037109e-01,9.4869446754455566e-01,Alt-lite
3,UCl3RCEtooHD5bhPCHJw3btA,1.4336268901824951e+00,6.8366008996963501e-01,Alt-lite
4,UCbukZErY9SjdGTHabMzY7qg,0.0000000000000000e+00,0.0000000000000000e+00,Alt-lite
...,...,...,...,...
285,UCQZ1lsBP6eMCQXxk41XpMew,5.8137202262878418e-01,5.8634036779403687e-01,Intellectual Dark Web
286,UCmVzhZ-0vh6sjLKHKGrGMRg,NaN,NaN,Intellectual Dark Web
287,UCSiYkMylC3F_eQfIodMHkOw,7.4698740243911743e-01,7.7780318260192871e-01,Intellectual Dark Web
288,UCZViJTyN0OXTTuPQ2mimNWw,2.0914123058319092e+00,1.9150885343551636e+00,Intellectual Dark Web


In [48]:
#counting the nomber of Nan values
nb_Nan=len(df_cluster_interest[df_cluster_interest.isnull().any(axis=1)])
print("there are", nb_Nan, "Nan values which corresponds to", nb_Nan*100/len(df_cluster_interest), "% of the dataframe")

there are 134 Nan values which corresponds to 46.206896551724135 % of the dataframe


In [51]:
df_cluster_interest[df_cluster_interest.isnull().any(axis=1)]

,channel_id,pagerank,pagerank_2018,Category
1,UCso6oQ3qy7z61Fr8b0WNBMw,NaN,NaN,Alt-lite
5,UC7tlcXIInkPG5aCkWB9lZOQ,NaN,NaN,Alt-lite
6,UCSFVD7Xfhn7sJY8LAIQmH8Q,NaN,NaN,Alt-lite
7,UCtLAZi2CZmDgwgQ51XpqcDQ,NaN,NaN,Alt-lite
11,UCDhuG96y6kUT7tcFgHGj1Ug,NaN,NaN,Alt-lite
...,...,...,...,...
267,UCAFa0zt3iJtEki2dotOfvrg,NaN,NaN,Intellectual Dark Web
271,UCRZsTalc2L7VjRJNzHeN4vg,NaN,NaN,Intellectual Dark Web
277,UCGyBhZvqlWjGZHNHrBzj2vQ,NaN,NaN,Intellectual Dark Web
284,UCGMCH9ZQRfLT3qQpZIqGPzg,NaN,NaN,Intellectual Dark Web


In [53]:
df_cluster_interest[df_cluster_interest['pagerank_2018'].isnull()]

,channel_id,pagerank,pagerank_2018,Category
1,UCso6oQ3qy7z61Fr8b0WNBMw,NaN,NaN,Alt-lite
5,UC7tlcXIInkPG5aCkWB9lZOQ,NaN,NaN,Alt-lite
6,UCSFVD7Xfhn7sJY8LAIQmH8Q,NaN,NaN,Alt-lite
7,UCtLAZi2CZmDgwgQ51XpqcDQ,NaN,NaN,Alt-lite
11,UCDhuG96y6kUT7tcFgHGj1Ug,NaN,NaN,Alt-lite
...,...,...,...,...
267,UCAFa0zt3iJtEki2dotOfvrg,NaN,NaN,Intellectual Dark Web
271,UCRZsTalc2L7VjRJNzHeN4vg,NaN,NaN,Intellectual Dark Web
277,UCGyBhZvqlWjGZHNHrBzj2vQ,NaN,NaN,Intellectual Dark Web
284,UCGMCH9ZQRfLT3qQpZIqGPzg,NaN,NaN,Intellectual Dark Web


In [57]:
#checking if rows where pagerank is Nan are also rows where pagerank_2018 is Nan
(df_cluster_interest[df_cluster_interest['pagerank'].isnull()]).equals(df_cluster_interest[df_cluster_interest['pagerank_2018'].isnull()])

True

In [59]:
#we choose to discard these rows as they don't bring new information
#dropping NaN values
df_cluster_interest.dropna(inplace=True)
nb_Nan=len(df_cluster_interest[df_cluster_interest.isnull().any(axis=1)])
print("there are", nb_Nan, "Nan values which corresponds to", nb_Nan*100/len(df_cluster_interest), "% of the dataframe")

there are 0 Nan values which corresponds to 0.0 % of the dataframe


In [61]:
df_cluster_interest['Category'].value_counts()

Alt-lite                 79
Intellectual Dark Web    57
Alt-right                20
Name: Category, dtype: int64

In [75]:
#converting our values into floats as they were under the string format
df_cluster_interest['pagerank']=df_cluster_interest['pagerank'].astype('float')
df_cluster_interest['pagerank_2018']=df_cluster_interest['pagerank_2018'].astype('float')

In [76]:
df_temp=df_cluster_interest.drop(['channel_id'], axis=1)
df_cluster_interest.groupby(by='Category').mean()

,pagerank,pagerank_2018
Category,,
Alt-lite,4.854888,6.121196
Alt-right,1.105379,1.435813
Intellectual Dark Web,3.202381,4.015690
